# 1. 전처리된 파일 불러오기

In [2]:
import pandas as pd

In [3]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [4]:
X_train.shape, y_train.shape, X_test.shape

((891, 22), (891, 1), (418, 22))

### 2. 앙상블 모델 사용하기
- 머신러닝에서 성능이 좋은 모델
- 여러개의 DecisionTree 를 사용하는 모델

### 2.1 RandomForest 모델 사용

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
# 모델생성
forest_model = RandomForestClassifier()

### 2.2 교차검증

In [7]:
from sklearn.model_selection import cross_val_score

In [8]:
# cv=5 평균 값 확인
cv_result = cross_val_score(forest_model, X_train, y_train, cv=5).mean()
cv_result

C:\Users\gjaischool1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\gjaischool1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\gjaischool1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\gjaischool1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarni

0.8047140794677045

In [9]:
import warnings
warnings.filterwarnings('ignore')

### 3. 학습

In [10]:
forest_model.fit(X_train, y_train)

RandomForestClassifier()

### 4. 평가

In [11]:
# 학습한 모델로 예측
pred = forest_model.predict(X_test)

# 평가 결과 답안지 불러오기
sub = pd.read_csv('./data/titanic/gender_submission.csv')

# 평가 결과 답안지에 예측한 값 넣기
sub['Survived']=pred

# 예측한 값 내보내기 ==> kaggle 에 업로드
sub.to_csv('bbb_submmision03.csv', index=False)

In [12]:
# 하이퍼 파라미터 튜닝
# n_estimators를 50~250 까지 바꿔가면서 결과 확인
for i in range(50, 80):
    forest_model1 = RandomForestClassifier(n_estimators = i)
    # 교차검증으로 평가
    result = cross_val_score(forest_model1, X_train, y_train, cv=5).mean()
    print(f'n_estimators :,{i}, score:{result}')

# n_estimators :,51, score:0.8125855250768941

n_estimators :,50, score:0.8069612704789405
n_estimators :,51, score:0.7968802962776976
n_estimators :,52, score:0.8036155922415418
n_estimators :,53, score:0.8002385286548239
n_estimators :,54, score:0.8002259745150964
n_estimators :,55, score:0.795744146632352
n_estimators :,56, score:0.8036155922415416
n_estimators :,57, score:0.806973824618668
n_estimators :,58, score:0.8092272926997678
n_estimators :,59, score:0.7991086560793422
n_estimators :,60, score:0.8069612704789405
n_estimators :,61, score:0.8024731655263324
n_estimators :,62, score:0.8081099742640134
n_estimators :,63, score:0.799114933149206
n_estimators :,64, score:0.8013621241604418
n_estimators :,65, score:0.8002448057246877
n_estimators :,66, score:0.7957504237022158
n_estimators :,67, score:0.8024919967359235
n_estimators :,68, score:0.8035967610319503
n_estimators :,69, score:0.7957315924926245
n_estimators :,70, score:0.803603038101814
n_estimators :,71, score:0.808097420124286
n_estimators :,72, score:0.8103383340

In [13]:
# 최적의 하이퍼파라미터 조합을 찾기 위해서는 여러개의 for 문이 필요하다. => 복잡, 작업이 번거롭다.
# 한번의 여러개의 하이퍼 파라미터를 튜닝해주는 함수 사용 => GridSearchCV

In [14]:
# 1. 하이퍼 파라미터 튜닝
# GridSearchCV(모델, 사용할 하이퍼파라미터, cv) : 하이퍼파라미터 튜닝 + 교차검증

# 1.관심(궁금한)있는 하이퍼파라미터 지정
# n_estimators = [50,70,90,110,130,150] : 나무의 개수
# max_depth = [3,4,5,6,7,8] : 나무의 최대 깊이
# max_features = [0.3,0.4,0.5,0.6,0.7] : 각 노드에서 고려할 특성의 개수

# 2. 파라미터를 딕셔너리 형태로 저장 key : 사용할 하이퍼 파라미터, value : 범위설정
param = {'n_estimators' : [50,70,90,110,130,150],
         'max_depth' : [3,4,5,6,7,8],
         'max_features' : [0.3,0.4,0.5,0.6,0.7]}

In [15]:
# GridSearchCV() 불러오기
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(RandomForestClassifier(), param, cv=5)

In [16]:
# 2. 학습
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8],
                         'max_features': [0.3, 0.4, 0.5, 0.6, 0.7],
                         'n_estimators': [50, 70, 90, 110, 130, 150]})

In [17]:
# 3. 최적의 파라미터 조합 찾기
grid.best_params_

{'max_depth': 7, 'max_features': 0.7, 'n_estimators': 50}

In [18]:
# 4. 최적의 파라미터 일 때 best_score
grid.best_score_

0.837266963781307

In [19]:
# 자세하게 찾아보기
param = {'n_estimators' : [64,66,68,70,72,74],
         'max_depth' : [5,6,7,8,9,10],
         'max_features' : [0.6,0.7,0.8,0.9]}

In [20]:
grid1 = GridSearchCV(RandomForestClassifier(), param, cv=5)

In [24]:
grid1.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 6, 7, 8, 9, 10],
                         'max_features': [0.6, 0.7, 0.8, 0.9],
                         'n_estimators': [64, 66, 68, 70, 72, 74]})

In [25]:
grid1.best_params_

{'max_depth': 9, 'max_features': 0.6, 'n_estimators': 68}

In [26]:
grid.best_score_

0.837266963781307

In [28]:
# 'max_depth': 9, 'max_features': 0.6, 'n_estimators': 68 ==> 베스트 모델 생성
forest_model1 = RandomForestClassifier(max_depth=9, max_features=0.6, n_estimators=68)
forest_model1.fit(X_train, y_train)

RandomForestClassifier(max_depth=9, max_features=0.6, n_estimators=68)

In [29]:
# 학습한 모델로 예측
pred = forest_model.predict(X_test)

# 평가 결과 답안지 불러오기
sub = pd.read_csv('./data/titanic/gender_submission.csv')

# 평가 결과 답안지에 예측한 값 넣기
sub['Survived']=pred

# 예측한 값 내보내기 ==> kaggle 에 업로드
sub.to_csv('bbb_submmision04.csv', index=False)